<a href="https://colab.research.google.com/github/shahabday/BAMline4CT/blob/main/notebooks/development/Merge_fo_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### if you are running on google colab run the following cells :

1. clone the repository
2. import config files




Create a GitHub Personal Access Token (PAT):

Go to your GitHub settings → Developer Settings → Personal Access Tokens → Generate new token (classic or fine-grained).

Choose scopes like repo (if using classic) or customize access if using fine-grained.

Copy and store the token safely.

In [8]:
from getpass import getpass

# Securely enter your GitHub token (it will be hidden)
git_token = getpass('Enter your GitHub token: ')

# Replace with your repo and username
repo_url = f"https://{git_token}@github.com/andandandand/jaguars.git"

# Clone the repository
!git clone $repo_url


Enter your GitHub token: ··········
fatal: destination path 'jaguars' already exists and is not an empty directory.


In [1]:
import os
import json


In [3]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')



drive_base_path = "/content/drive/MyDrive"
with open("jaguars/config/colab_config.json") as f:
    cfg = json.load(f)

# Step 4: Build absolute paths
project_root = os.path.join(drive_base_path, cfg["project_subdir"])
image_dir = os.path.join(project_root, cfg["image_dir"])
dataset_path = os.path.join(project_root, cfg["dataset_path"])
output_dir = os.path.join(project_root, cfg["output_dir"])

print("Image Dir:", image_dir)
print("Dataset Path:", dataset_path)


Mounted at /content/drive
Image Dir: /content/drive/MyDrive/animal-segmentation/images/full
Dataset Path: /content/drive/MyDrive/animal-segmentation/datasets/dataset_v1.pkl


# Installing and importing necessary tools for Fifty One

In [4]:
!pip install fiftyone

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.0/943.0 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 4.2 MB/s eta 0:00:0

In [2]:
import os
import fiftyone as fo
from pathlib import Path
import fiftyone.zoo as foz
import fiftyone.brain as fob
import fiftyone.utils.image as foui
from PIL import Image
import numpy as np
import sys


In [ ]:
def read_dataset(path, base_dir=None):
    dataset = fo.Dataset.from_dir(
        dataset_dir=str(path),
        dataset_type=fo.types.FiftyOneDataset,
        rel_dir=base_dir if base_dir else None # if base_dir exists add otherwise do nothing
    )
    return dataset

In [ ]:
def merge_datasets(dataset_dirs,merged_dataset_name = "meged_dataset", base_dir=None):
  """
  this function merges several datasets .

  """

  datasets = []
  for dataset_path in dataset_dirs:
      ds = read_dataset(dataset_path, base_dir=base_dir)
      datasets.append(ds)


  try:
      if fo.dataset_exists(merged_dataset_name) :
        fo.delete_dataset(merged_dataset_name)
  except NameError:
      pass

  merged_dataset = fo.Dataset(merged_dataset_name)
  for ds in datasets:
      merged_dataset.add_samples([s.copy() for s in ds])


  if base_dir:
    merged_dataset.info["base_dir"] = str(base_dir)

  return merged_dataset


In [61]:


base_dir = Path('/content/drive/MyDrive/DataScience/Jaguars_Project/images/raw_images/')
# Paths to the saved dataset directories in your drive
dataset_dirs = [
   Path('/content/drive/MyDrive/DataScience/Jaguars_Project/datasets/dataset_v2/g_dino_all_{}'.format(batch_number)),
    Path('/content/drive/MyDrive/DataScience/Jaguars_Project/datasets/dataset_v2/g_dino_all_{}'.format(batch_number+1)),
    Path('/content/drive/MyDrive/DataScience/Jaguars_Project/datasets/dataset_v2/g_dino_all_{}'.format(batch_number+2))
]

In [62]:
merged_dataset = merge_datasets(dataset_dirs,merged_dataset_name = "meged_dataset", base_dir=base_dir)

Importing samples...


INFO:fiftyone.utils.data.importers:Importing samples...


 100% |█████████████████| 100/100 [15.3ms elapsed, 0s remaining, 6.5K samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 100/100 [15.3ms elapsed, 0s remaining, 6.5K samples/s]      


Importing samples...


...

INFO:eta.core.utils: 100% |█████████████████| 100/100 [253.8ms elapsed, 0s remaining, 407.7 samples/s] 


 100% |█████████████████| 100/100 [179.8ms elapsed, 0s remaining, 556.1 samples/s]     


INFO:eta.core.utils: 100% |█████████████████| 100/100 [179.8ms elapsed, 0s remaining, 556.1 samples/s]     


 100% |█████████████████| 100/100 [196.2ms elapsed, 0s remaining, 509.6 samples/s]     


INFO:eta.core.utils: 100% |█████████████████| 100/100 [196.2ms elapsed, 0s remaining, 509.6 samples/s]     


In [63]:
merged_dataset.info

{'base_dir': '/content/drive/MyDrive/DataScience/Jaguars_Project/images/raw_images'}

In [64]:
for sample in merged_dataset:
  pass


In [65]:
sample

<Sample: {
    'id': '67ebbb13b58da224b2f7481c',
    'media_type': 'image',
    'filepath': '/content/drive/MyDrive/DataScience/Jaguars_Project/images/raw_images/Alira/A_Alira_26.jpg',
    'tags': [],
    'metadata': None,
    'created_at': datetime.datetime(2025, 4, 1, 10, 8, 19, 526000),
    'last_modified_at': datetime.datetime(2025, 4, 1, 10, 8, 19, 526000),
    'ground_truth': <Classification: {
        'id': '67ebbb13b58da224b2f747b7',
        'tags': [],
        'label': 'Alira',
        'confidence': None,
        'logits': None,
    }>,
    'prediction': <Detections: {
        'detections': [
            <Detection: {
                'id': '67ebbb13b58da224b2f747b8',
                'attributes': {},
                'tags': [],
                'label': "jaguar's whole body",
                'bounding_box': [
                    0.4771029713114754,
                    0.4316882183908046,
                    0.21196465163934425,
                    0.1272772988505747,
          